# Дашборд для Яндекс.Дзен
==============================================================================================================

## Описание проекта

Вы работаете аналитиком в Яндекс.Дзене. Почти всё ваше время занимает анализ пользовательского взаимодействия с карточками статей.

Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия».

Пользователей системы характеризует возрастная категория. Скажем, «26-30» или «45+».

Есть три способа взаимодействия пользователей с системой:
- Карточка отображена для пользователя (show);
- Пользователь кликнул на карточку (click);
- Пользователь просмотрел статью карточки (view).

Каждую неделю начинающие менеджеры Денис и Валерия задают вам одни и те же вопросы:
- Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
- Как много карточек генерируют источники с разными темами?
- Как соотносятся темы карточек и темы источников?
На шестую неделю работы вы решаете, что процесс пора автоматизировать. Для Дениса и Валерии нужно сделать дашборд.
Дашборд будет основываться на пайплайне, который будет брать данные из таблицы, в которых хранятся сырые данные, трансформировать данные и укладывать их в агрегирующую таблицу. Пайплайн будет разработан для вас дата-инженерами.

*Краткое ТЗ:*

**Бизнес-задача:** 
- анализ взаимодействия пользователей с карточками Яндекс.Дзен;
- Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;
- Кто будет основным пользователем дашборда: менеджеры по анализу контента;

**Состав данных для дашборда:**
- История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
- Разбивка событий по темам источников;
- Таблица соответствия тем источников темам карточек;
- По каким параметрам данные должны группироваться:
- Дата и время;
- Тема карточки;
- Тема источника;
- Возрастная группа;

**Характер данных:**

- История событий по темам карточек — абсолютные величины с разбивкой по минутам;
- Разбивка событий по темам источников — относительные величины (% событий);
- Соответствия тем источников темам карточек - абсолютные величины;

**Важность:** все графики имеют равную важность;
- Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);
- База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД zen;
- Частота обновления данных: один раз в сутки, в полночь по UTC;
- Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде (макет дашборда):

## Изучение данных

In [1]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

In [2]:
query = '''
           SELECT *
           FROM dash_visits
        '''
dash_visits = pd.io.sql.read_sql(query, con = engine)
dash_visits.sample(7)

,record_id,item_topic,source_topic,age_segment,dt,visits
15194,1055791,Общество,Кино,41-45,2019-09-24 18:54:00,3
24140,1064737,Россия,Здоровье,45+,2019-09-24 18:57:00,1
22427,1063024,Путешествия,Россия,26-30,2019-09-24 18:32:00,1
27871,1068468,Туризм,Знаменитости,45+,2019-09-24 18:58:00,1
11179,1051776,Красота,История,18-25,2019-09-24 18:32:00,1
9222,1049819,Искусство,Сделай сам,36-40,2019-09-24 19:00:00,1
26722,1067319,Скандалы,Здоровье,36-40,2019-09-24 18:58:00,3


In [3]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
record_id       30745 non-null int64
item_topic      30745 non-null object
source_topic    30745 non-null object
age_segment     30745 non-null object
dt              30745 non-null datetime64[ns]
visits          30745 non-null int64
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


✅ Отчет по логам содержит 30745 строк, с данными идентификации карточки, её тематики, рекомендуемый возраст пользователей, время действия пользователя с карточкой и количеством действий в определенный момент времени.

In [4]:
#посмотрим на дубликаты и пропуски
print ('Количество дубликатов:', dash_visits.duplicated().sum())
print()
print('Количество пропусков:')
print(dash_visits.isnull().sum())

Количество дубликатов: 0

Количество пропусков:
record_id       0
item_topic      0
source_topic    0
age_segment     0
dt              0
visits          0
dtype: int64


✅ В отчете нет ни пропусков, ни дубликатов.

Исследуем поподробнее

In [5]:
dash_visits.dt.describe()

count                   30745
unique                     17
top       2019-09-24 18:58:00
freq                     3383
first     2019-09-24 18:28:00
last      2019-09-24 19:00:00
Name: dt, dtype: object

✅ Взаимодействия пользователей с карточками представлено 17ю временным моментами,  в отрезке 32 минут. Самое популярное время 18:58.

In [6]:
display(dash_visits.item_topic.unique())
display(dash_visits.item_topic.describe())

array(['Деньги', 'Дети', 'Женская психология', 'Женщины', 'Здоровье',
       'Знаменитости', 'Интересные факты', 'Искусство', 'История',
       'Красота', 'Культура', 'Наука', 'Общество', 'Отношения',
       'Подборки', 'Полезные советы', 'Психология', 'Путешествия',
       'Рассказы', 'Россия', 'Семья', 'Скандалы', 'Туризм', 'Шоу', 'Юмор'],
      dtype=object)

count         30745
unique           25
top       Отношения
freq           1536
Name: item_topic, dtype: object

In [7]:
dash_visits.groupby('item_topic')['visits'].sum().sort_values(ascending=False).head(5)

item_topic
Наука               21736
Отношения           20666
Интересные факты    19942
Общество            19640
Подборки            17772
Name: visits, dtype: int64

✅ За установленный временной отрезок пользователи провели действие с карточками, отражающие 25 уникальных тем. Данные темы определяет автор карточки. Таким образом самой популярной темой в отчете стала тема - Отношения. Однако, алгоритмы Дзена также анализируют содержание всех публикаций и на основании этого анализа каждой публикации присваиваются определенные интересы. Так определяется источник (тема) тематической ленты. Таким образом, по количеству визитов самым популярным источником стала тема карточек - Наука. 

In [8]:
display(dash_visits.source_topic.unique())
display(dash_visits.source_topic.describe())

array(['Авто', 'Деньги', 'Дети', 'Еда', 'Здоровье', 'Знаменитости',
       'Интерьеры', 'Искусство', 'История', 'Кино', 'Музыка', 'Одежда',
       'Полезные советы', 'Политика', 'Психология', 'Путешествия',
       'Ремонт', 'Россия', 'Сад и дача', 'Сделай сам',
       'Семейные отношения', 'Семья', 'Спорт', 'Строительство',
       'Технологии', 'Финансы'], dtype=object)

count                  30745
unique                    26
top       Семейные отношения
freq                    1822
Name: source_topic, dtype: object

In [9]:
dash_visits.groupby('source_topic')['visits'].sum().sort_values(ascending=False).head(5)

source_topic
Семейные отношения    33309
Россия                29831
Полезные советы       27412
Путешествия           24124
Знаменитости          23945
Name: visits, dtype: int64

✅ Тема семейных отнношений была наиболее популярна среди пользователей.

In [10]:
display(dash_visits.age_segment.unique())
display(dash_visits.age_segment.describe())

array(['18-25', '26-30', '31-35', '36-40', '41-45', '45+'], dtype=object)

count     30745
unique        6
top       18-25
freq       7056
Name: age_segment, dtype: object

In [11]:
dash_visits.groupby('age_segment')['visits'].sum().sort_values(ascending=False).head(5)

age_segment
18-25    131010
26-30     60311
31-35     53763
36-40     38166
41-45     16263
Name: visits, dtype: int64

In [18]:
# сохраняю в csv
dash_visits.to_csv('dash_visits_tabl.csv', index = False)

Ссылка на дашборд в Tableau online - https://prod-uk-a.online.tableau.com/#/site/skb/views/Avto/Dashboard1?:iid=3

Ссылка на презентацию - https://disk.yandex.ru/i/yUYQWs0mTos87w